In [1]:
!pip install praw
!pip install transformers
!pip install sentencepiece
!pip install torch
!pip install --upgrade asyncpraw
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 9.6/13.8 MB 128.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please 

In [2]:
import asyncpraw
import asyncio
from transformers import pipeline
from transformers import BartTokenizer

# Setting up Reddit instance
async def setup_reddit():
    reddit = asyncpraw.Reddit(client_id = "k_-waUAoYeDwXPOXSw4RoA" ,
                        client_secret = "Ggq-0JiNR8GnzT9eQsfhiW80KrD6nA",
                        user_agent = "personal project"
                        )
    return reddit

# Fetching posts from a subreddit
async def fetch_posts(reddit, subreddit_name, limit=10):
    subreddit = await reddit.subreddit(subreddit_name)
    posts = subreddit.new(limit=limit)
    return posts

async def fetch_comments(post):
    await post.load()
    comments = post.comments.list()

    while True:
        more_comments = [comment for comment in comments if isinstance(comment, asyncpraw.models.MoreComments)]
        if not more_comments:
            break
        for more_comment in more_comments:
            await more_comment.replace_more(limit=32)
            comments = post.comments.list()

    return comments


def preprocess_text(comments):
    text = ""
    for comment in comments:
        if isinstance(comment, asyncpraw.models.Comment):
            text += comment.body + " "
            if len(text.split()) > 1024:
                break
    return text.strip()



def summarize_text(text):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer=tokenizer)

    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt")

    summary_ids = summarizer.model.generate(
        inputs.input_ids,
        max_length=130,
        min_length=30,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary
async def main():
    reddit = await setup_reddit()
    posts = await fetch_posts(reddit, "anime")

    async for post in posts:
        print(f"Title: {post.title}")
        comments = await fetch_comments(post)

        if comments:
            text = preprocess_text(comments)

            if text:
                summary = summarize_text(text)
                print(f"Summary: {summary}")
            else:
                print("No comments found.")
        else:
            print("No comments found.")

        print("\n")

    await reddit.close()


await main()

Title: Anyone know the name of soundtrack? [Naruto]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Summary: If you ever find yourself needing anime related help, here are a few resources to save you a LOT of time. Have an anime related question? Try our [daily question/recommendation thread!


Title: Good anime with strong/ supernatural female lead character, like Darling in the Franxx, Dandadan, vampire movie series?


Device set to use cpu


Summary: The users of this subreddit came up with [a chart of anime recommendations for newcomers and veterans] and [an awesome longer flowchart] Maybe you can find something there that you'll like. You might also find our [Recommendation Wiki] or [daily thread] helpful.


Title: Quetzalcoatl drops the weights [Fate/Grand Order Absolute Demonic Front: Babylonia]


Device set to use cpu


Summary: Babylonia sound design was absolute peak. One of my favorite servants, she is so overpowered she had to be nerf hard in the chapter and she just loves humanity. If only her NP was this cool in-game.


